In [1]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# ============================================================================
# DATA PREPARATION - HIERARCHICAL CITY-DISTRICT STRUCTURE
# ============================================================================

# Define cities and their districts
city_districts = {
    "Istanbul": [
        "Istanbul_Europe_North", "Istanbul_Europe_Central", "Istanbul_Europe_South",
        "Istanbul_Asia_North", "Istanbul_Asia_Central"
    ],
    "Bursa": ["Bursa_Central", "Bursa_Industrial"],
    "Izmir": ["Izmir_Central", "Izmir_Karsiyaka", "Izmir_Konak"],
    "Kocaeli": ["Kocaeli_Central", "Kocaeli_Industrial"],
    "Balikesir": ["Balikesir_Central"],
    "Canakkale": ["Canakkale_Central"],
    "Sakarya": ["Sakarya_Central"],
    "Tekirdag": ["Tekirdag_Central"],
    "Aydin": ["Aydin_Central"],
    "Denizli": ["Denizli_Central"],
    "Manisa": ["Manisa_Central"],
    "Mugla": ["Mugla_Central"],
    "Usak": ["Usak_Central"]
}

# Flatten districts list and create city mapping
all_districts = []
district_to_city = {}
for city, districts in city_districts.items():
    for district in districts:
        all_districts.append(district)
        district_to_city[district] = city


# ============================================================================
# POPULATION AND RISK DATA (DISTRIBUTED TO DISTRICTS)
# ============================================================================

# Original city data
city_base_data = {
    "Istanbul": {"population": 16047000, "risk": 10},
    "Bursa": {"population": 2116000, "risk": 8},
    "Izmir": {"population": 3120000, "risk": 9},
    "Kocaeli": {"population": 2100000, "risk": 10},
    "Balikesir": {"population": 1260000, "risk": 6},
    "Canakkale": {"population": 560000, "risk": 7},
    "Sakarya": {"population": 1085000, "risk": 8},
    "Tekirdag": {"population": 1150000, "risk": 4},
    "Aydin": {"population": 1150000, "risk": 5},
    "Denizli": {"population": 1060000, "risk": 4},
    "Manisa": {"population": 1470000, "risk": 6},
    "Mugla": {"population": 1050000, "risk": 3},
    "Usak": {"population": 375000, "risk": 1}
}

# Population distribution within cities (proportions)
district_population_share = {
    # Istanbul districts
    "Istanbul_Europe_North": 0.25,
    "Istanbul_Europe_Central": 0.20,
    "Istanbul_Europe_South": 0.20,
    "Istanbul_Asia_North": 0.20,
    "Istanbul_Asia_Central": 0.15,
    
    # Bursa districts
    "Bursa_Central": 0.70,
    "Bursa_Industrial": 0.30,
    
    # Izmir districts
    "Izmir_Central": 0.40,
    "Izmir_Karsiyaka": 0.35,
    "Izmir_Konak": 0.25,
    
    # Kocaeli districts
    "Kocaeli_Central": 0.60,
    "Kocaeli_Industrial": 0.40,
    
    # Single-district cities
    "Balikesir_Central": 1.0,
    "Canakkale_Central": 1.0,
    "Sakarya_Central": 1.0,
    "Tekirdag_Central": 1.0,
    "Aydin_Central": 1.0,
    "Denizli_Central": 1.0,
    "Manisa_Central": 1.0,
    "Mugla_Central": 1.0,
    "Usak_Central": 1.0
}

# Calculate district-level data
district_data = {}
for district in all_districts:
    city = district_to_city[district]
    city_pop = city_base_data[city]["population"]
    city_risk = city_base_data[city]["risk"]
    
    district_data[district] = {
        "population": int(city_pop * district_population_share[district]),
        "risk": city_risk,
        "city": city
    }

# ============================================================================
# DISTANCE MATRIX (SIMPLIFIED FOR DISTRICTS)
# ============================================================================

# Approximate coordinates for districts (latitude, longitude)
district_coords = {
    "Istanbul_Europe_North": (41.08, 28.95),
    "Istanbul_Europe_Central": (41.01, 28.98),
    "Istanbul_Europe_South": (40.99, 28.80),
    "Istanbul_Asia_North": (41.02, 29.12),
    "Istanbul_Asia_Central": (40.90, 29.20),
    
    "Bursa_Central": (40.18, 29.07),
    "Bursa_Industrial": (40.22, 29.12),
    
    "Izmir_Central": (38.42, 27.13),
    "Izmir_Karsiyaka": (38.47, 27.11),
    "Izmir_Konak": (38.38, 27.14),
    
    "Kocaeli_Central": (40.77, 29.91),
    "Kocaeli_Industrial": (40.82, 29.95),
    
    "Balikesir_Central": (39.65, 27.88),
    "Canakkale_Central": (40.16, 26.41),
    "Sakarya_Central": (40.78, 30.40),
    "Tekirdag_Central": (40.98, 27.51),
    "Aydin_Central": (37.84, 27.84),
    "Denizli_Central": (37.78, 29.09),
    "Manisa_Central": (38.62, 27.43),
    "Mugla_Central": (37.22, 28.37),
    "Usak_Central": (38.68, 29.41)
}

def calculate_distance(coord1, coord2):
    """Calculate approximate distance between two coordinates"""
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    # Simple Euclidean distance approximation in km multiplied by 1.4 to better estimate driving distances
    lat_diff = (lat2 - lat1) * 111.0
    lon_diff = (lon2 - lon1) * 111.0 * np.cos(np.radians((lat1 + lat2) / 2))
    return np.sqrt(lat_diff**2 + lon_diff**2) * 1.4

# Create distance matrix
distance_matrix = {}
for dist1 in all_districts:
    for dist2 in all_districts:
        if dist1 == dist2:
            distance_matrix[(dist1, dist2)] = 0
        else:
            distance_matrix[(dist1, dist2)] = calculate_distance(
                district_coords[dist1], 
                district_coords[dist2]
            )

# ============================================================================
# MODEL PARAMETERS
# ============================================================================

# Scenario Parameters
AFFECTED_RATIO = 0.25
COVERAGE_RATIO = 0.30
UNITS_PER_PERSON = 2.9375
DISTANCE_WEIGHT = 0.01
MIN_UTILIZATION = 0.50
MIN_DISTANCE = 20.0  
MIN_TOTALWAREHOUSE = 6
MIN_MARMARA = 3
MIN_AEGEAN = 2
MIN_NORTHWEST = 1
MIN_IST = 2

# Maximum warehouses per city
MAX_WAREHOUSES_PER_CITY = {
    "Istanbul": 4,  # Large megacity
    "Izmir": 2,     # Major city
    "Bursa": 1,     
    "Kocaeli": 1,   
    "Balikesir": 1,
    "Canakkale": 1,
    "Sakarya": 1,
    "Tekirdag": 1,
    "Aydin": 1,
    "Denizli": 1,
    "Manisa": 1,
    "Mugla": 1,
    "Usak": 1
}

# Warehouse specifications
warehouse_sizes = {
    'Small': {'capacity': 1250000, 'cost': 17},
    'Medium': {'capacity': 1500000, 'cost': 20},
    'Large': {'capacity': 2200000, 'cost': 25}
}

# Calculate demand for each district
for district in all_districts:
    pop = district_data[district]["population"]
    district_data[district]["demand"] = pop * AFFECTED_RATIO * COVERAGE_RATIO * UNITS_PER_PERSON

# Convert to dictionaries for Pyomo
demand_dict = {d: district_data[d]["demand"] for d in all_districts}
risk_dict = {d: district_data[d]["risk"] for d in all_districts}
capacity_dict = {s: warehouse_sizes[s]['capacity'] for s in warehouse_sizes}
cost_dict = {s: warehouse_sizes[s]['cost'] for s in warehouse_sizes}

print(f"Total demand across all districts: {sum(demand_dict.values()):,.0f} units")

# ============================================================================
# MODEL
# ============================================================================

model = pyo.ConcreteModel()

# Sets
model.CITIES = pyo.Set(initialize=list(city_districts.keys()))
model.DISTRICTS = pyo.Set(initialize=all_districts)
model.SIZES = pyo.Set(initialize=warehouse_sizes.keys())

# Parameters
model.demand = pyo.Param(model.DISTRICTS, initialize=demand_dict)
model.risk = pyo.Param(model.DISTRICTS, initialize=risk_dict)
model.distance = pyo.Param(model.DISTRICTS, model.DISTRICTS, initialize=distance_matrix)
model.capacity = pyo.Param(model.SIZES, initialize=capacity_dict)
model.cost = pyo.Param(model.SIZES, initialize=cost_dict)

# Decision Variables
model.y = pyo.Var(model.DISTRICTS, model.SIZES, within=pyo.Binary)
model.x = pyo.Var(model.DISTRICTS, model.DISTRICTS, within=pyo.Binary)

# ============================================================================
# OBJECTIVE FUNCTION
# ============================================================================

def objective_rule(model):
    build_cost = sum(model.cost[s] * model.y[i, s] 
                    for i in model.DISTRICTS for s in model.SIZES)
    transport_risk_cost = DISTANCE_WEIGHT * sum(
        model.distance[i, j] * model.risk[j] * model.x[i, j] 
        for i in model.DISTRICTS for j in model.DISTRICTS
    )
    return build_cost + transport_risk_cost

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# ============================================================================
# CONSTRAINTS
# ============================================================================

# 1. Each district is served by exactly one warehouse
def assignment_rule(model, j):
    return sum(model.x[i, j] for i in model.DISTRICTS) == 1

model.assignment_con = pyo.Constraint(model.DISTRICTS, rule=assignment_rule)

# 2. Warehouse capacity constraint
def capacity_rule(model, i):
    assigned_demand = sum(model.demand[j] * model.x[i, j] for j in model.DISTRICTS)
    warehouse_capacity = sum(model.capacity[s] * model.y[i, s] for s in model.SIZES)
    return assigned_demand <= warehouse_capacity

model.capacity_con = pyo.Constraint(model.DISTRICTS, rule=capacity_rule)

# 3. Assignment enabler constraint
def enabler_rule(model, i, j):
    return model.x[i, j] <= sum(model.y[i, s] for s in model.SIZES)

model.enabler_con = pyo.Constraint(model.DISTRICTS, model.DISTRICTS, rule=enabler_rule)

# 4. Unique warehouse size constraint
def unique_size_rule(model, i):
    return sum(model.y[i, s] for s in model.SIZES) <= 1

model.unique_size_con = pyo.Constraint(model.DISTRICTS, rule=unique_size_rule)

# 5. Minimum utilization constraint
def utilization_rule(model, i):
    assigned_demand = sum(model.demand[j] * model.x[i, j] for j in model.DISTRICTS)
    warehouse_capacity = sum(model.capacity[s] * model.y[i, s] for s in model.SIZES)
    return assigned_demand >= MIN_UTILIZATION * warehouse_capacity

model.utilization_con = pyo.Constraint(model.DISTRICTS, rule=utilization_rule)

# 6. City-level maximum warehouse constraints
def max_warehouses_per_city_rule(model, city):
    city_districts = [d for d in model.DISTRICTS if district_to_city[d] == city]
    return sum(model.y[d, s] for d in city_districts for s in model.SIZES) <= MAX_WAREHOUSES_PER_CITY[city]

model.max_warehouses_city_con = pyo.Constraint(model.CITIES, rule=max_warehouses_per_city_rule)

# 7. Minimum warehouses for Istanbul (at least 2)
def min_istanbul_warehouses_rule(model):
    istanbul_districts = [d for d in model.DISTRICTS if district_to_city[d] == "Istanbul"]
    return sum(model.y[d, s] for d in istanbul_districts for s in model.SIZES) >= MIN_IST

model.min_istanbul_warehouses_con = pyo.Constraint(rule=min_istanbul_warehouses_rule)

# 8. Distance-based separation constraints 
def distance_separation_rule(model, i, j, s1, s2):
    if i != j and pyo.value(model.distance[i, j]) < MIN_DISTANCE:
        return model.y[i, s1] + model.y[j, s2] <= 1
    else:
        return pyo.Constraint.Skip

model.distance_separation_con = pyo.Constraint(
    model.DISTRICTS, model.DISTRICTS, model.SIZES, model.SIZES,
    rule=distance_separation_rule
)

# 9. Maximum service distance constraints
def max_distance_rule(model, i, j):
    # For high-risk areas 
    if model.risk[j] == 10:
        return model.distance[i, j] * model.x[i, j] <= 90
    elif model.risk[j] == 9:
        return model.distance[i, j] * model.x[i, j] <= 120
    elif model.risk[j] == 8:
        return model.distance[i, j] * model.x[i, j] <= 150
    elif model.risk[j] == 7:
        return model.distance[i, j] * model.x[i, j] <= 200
    # For all other areas
    else:
        return model.distance[i, j] * model.x[i, j] <= 230

model.max_distance_con = pyo.Constraint(model.DISTRICTS, model.DISTRICTS, rule=max_distance_rule)

# 10. Ensure minimum total warehouses
def min_total_warehouses_rule(model):
    return sum(model.y[i, s] for i in model.DISTRICTS for s in model.SIZES) >= MIN_TOTALWAREHOUSE

model.min_total_warehouses_con = pyo.Constraint(rule=min_total_warehouses_rule)

# 11a. Marmara Region constraint
def marmara_coverage_rule(model):
    marmara_districts = [d for d in model.DISTRICTS if district_to_city[d] in 
                        ["Istanbul", "Bursa", "Kocaeli", "Sakarya", "Tekirdag"]]
    return sum(model.y[d, s] for d in marmara_districts for s in model.SIZES) >= MIN_MARMARA

model.marmara_coverage_con = pyo.Constraint(rule=marmara_coverage_rule)

# 11b. Aegean Region constraint
def aegean_coverage_rule(model):
    aegean_districts = [d for d in model.DISTRICTS if district_to_city[d] in 
                       ["Izmir", "Manisa", "Aydin", "Denizli", "Mugla", "Usak"]]
    return sum(model.y[d, s] for d in aegean_districts for s in model.SIZES) >= MIN_AEGEAN

model.aegean_coverage_con = pyo.Constraint(rule=aegean_coverage_rule)

# 11c. Northwest Region constraint
def northwest_coverage_rule(model):
    northwest_districts = [d for d in model.DISTRICTS if district_to_city[d] in 
                          ["Balikesir", "Canakkale"]]
    return sum(model.y[d, s] for d in northwest_districts for s in model.SIZES) >= MIN_NORTHWEST

model.northwest_coverage_con = pyo.Constraint(rule=northwest_coverage_rule)
# ============================================================================
# SOLVE AND ANALYZE
# ============================================================================

print("\nSolving model...")
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=False)

# ============================================================================
# RESULTS ANALYSIS
# ============================================================================

print(f"\nSolver Status: {results.solver.status}")
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    total_cost = pyo.value(model.objective)
    fixed_cost = sum(pyo.value(model.cost[s]) * pyo.value(model.y[i, s]) 
                    for i in model.DISTRICTS for s in model.SIZES)
    
    print(f"Total Optimal Cost: {total_cost:.2f} million (unitless)")
    print(f"Fixed Cost Component: {fixed_cost:.2f} million (TL)")
    print(f"Transport Cost Component: {total_cost - fixed_cost:.2f} million (unitless)")
    
    # Warehouse locations by city
    print("\n" + "="*60)
    print("WAREHOUSE LOCATIONS BY CITY")
    print("="*60)
    
    city_warehouse_count = {}
    for city in model.CITIES:
        city_warehouses = []
        for district in model.DISTRICTS:
            if district_to_city[district] == city:
                for size in model.SIZES:
                    if pyo.value(model.y[district, size]) > 0.5:
                        city_warehouses.append((district, size))
        
        city_warehouse_count[city] = len(city_warehouses)
        if city_warehouses:
            print(f"\n{city} ({len(city_warehouses)} warehouses):")
            for district, size in city_warehouses:
                capacity = pyo.value(model.capacity[size])
                cost = pyo.value(model.cost[size])
                
                # Calculate utilization
                total_demand = sum(pyo.value(model.demand[j]) * pyo.value(model.x[district, j]) 
                                 for j in model.DISTRICTS)
                utilization = (total_demand / capacity) * 100
                
                print(f"  → {district}: {size} warehouse")
                print(f"     Cost: {cost} million TL, Capacity: {capacity:,} units")
                print(f"     Utilization: {utilization:.1f}% ({total_demand:,.0f} units)")
    
    # District assignments summary
    print(f"\n" + "="*60)
    print("DISTRICT ASSIGNMENT SUMMARY")
    print("="*60)
    
    total_districts_served = 0
    for i in model.DISTRICTS:
        if sum(pyo.value(model.y[i, s]) for s in model.SIZES) > 0.5:
            served_districts = []
            for j in model.DISTRICTS:
                if pyo.value(model.x[i, j]) > 0.5:
                    served_districts.append(j)
            
            total_districts_served += len(served_districts)
            size = [s for s in model.SIZES if pyo.value(model.y[i, s]) > 0.5][0]
            
            print(f"\nWarehouse at {i} ({size}) serves {len(served_districts)} districts:")
            for district in served_districts:
                demand = pyo.value(model.demand[district])
                distance = pyo.value(model.distance[i, district])
                risk = pyo.value(model.risk[district])
                print(f"  - {district}: {demand:,.0f} units, {distance:.0f}km, risk {risk}")
    
    # Summary statistics
    print(f"\n" + "="*60)
    print("SOLUTION SUMMARY")
    print("="*60)
    
    total_warehouses = sum(city_warehouse_count.values())
    total_demand = sum(pyo.value(model.demand[d]) for d in model.DISTRICTS)
    total_capacity = sum(pyo.value(model.capacity[s]) * pyo.value(model.y[i, s]) 
                        for i in model.DISTRICTS for s in model.SIZES)
    
    print(f"Total Warehouses Built: {total_warehouses}")
    print(f"Total System Demand: {total_demand:,.0f} units")
    print(f"Total System Capacity: {total_capacity:,.0f} units")
    print(f"Overall System Utilization: {(total_demand/total_capacity)*100:.1f}%")
    print(f"Districts Served: {len(model.DISTRICTS)}")
    
    # City-level warehouse distribution
    print(f"\nWarehouse Distribution by City:")
    for city, count in city_warehouse_count.items():
        max_allowed = MAX_WAREHOUSES_PER_CITY[city]
        print(f"  {city}: {count}/{max_allowed} warehouses")
    
    
    # Check minimum distance constraint
    violations = 0
    for i in model.DISTRICTS:
        for j in model.DISTRICTS:
            if i != j and pyo.value(model.distance[i, j]) < MIN_DISTANCE:
                wh_i = sum(pyo.value(model.y[i, s]) for s in model.SIZES)
                wh_j = sum(pyo.value(model.y[j, s]) for s in model.SIZES)
                if wh_i > 0.5 and wh_j > 0.5:
                    violations += 1
                    dist = pyo.value(model.distance[i, j])
                    print(f"⚠ Distance violation: {i} ↔ {j} = {dist:.1f}km < {MIN_DISTANCE}km")
    

else:
    print(f"Optimization failed: {results.solver.termination_condition}")

print(f"\n" + "="*60)
print("MODEL EXECUTION COMPLETE")
print("="*60)

Total demand across all districts: 7,169,630 units

Solving model...

Solver Status: ok
Total Optimal Cost: 181.75 million (unitless)
Fixed Cost Component: 113.00 million (TL)
Transport Cost Component: 68.75 million (unitless)

WAREHOUSE LOCATIONS BY CITY

Istanbul (2 warehouses):
  → Istanbul_Europe_South: Medium warehouse
     Cost: 20 million TL, Capacity: 1,500,000 units
     Utilization: 94.3% (1,414,142 units)
  → Istanbul_Asia_North: Large warehouse
     Cost: 25 million TL, Capacity: 2,200,000 units
     Utilization: 96.4% (2,121,213 units)

Izmir (1 warehouses):
  → Izmir_Central: Small warehouse
     Cost: 17 million TL, Capacity: 1,250,000 units
     Utilization: 80.9% (1,011,234 units)

Kocaeli (1 warehouses):
  → Kocaeli_Central: Small warehouse
     Cost: 17 million TL, Capacity: 1,250,000 units
     Utilization: 93.4% (1,167,877 units)

Canakkale (1 warehouses):
  → Canakkale_Central: Small warehouse
     Cost: 17 million TL, Capacity: 1,250,000 units
     Utilization: 5